In [1]:
import pandas as pd

import torch
from torch import nn
from torch import Tensor
import math
import configparser
from torch.utils.data import Dataset, DataLoader

from torch import cuda


In [2]:
file = open('Avg_data.csv','r')
df = pd.read_csv(file)
df

,Date,Day of the week [-],Temp [C],Humidity [%],Wind [m/s],Clouds [1/8],SUM - Entry [GWh/d],SUM - Exit [GWh/d],SUM - Exit t-1d [GWh/d],Storage t-1d [GWh/d],Storage [GWh/d]
0,01/01/2017,7.0,-2.078947,85.666667,2.817778,2.587273,466.017771,555.507468,555.507468,179.179566,179.179566
1,02/01/2017,1.0,-1.885965,83.711111,2.886667,2.647273,482.377191,615.189159,555.507468,179.179566,185.100405
2,03/01/2017,2.0,-1.631579,89.711111,3.633333,3.312727,507.121338,623.232658,615.189159,185.100405,218.470698
3,04/01/2017,3.0,0.229825,88.800000,5.257778,4.689091,484.296908,626.042006,623.232658,218.470698,207.183162
4,05/01/2017,4.0,-5.033333,80.111111,4.384444,3.974545,483.992000,689.796844,626.042006,207.183162,223.941467
...,...,...,...,...,...,...,...,...,...,...,...
1821,27/12/2021,3.0,-10.050877,85.044444,1.824444,1.736364,586.072921,851.060055,778.596491,116.434676,164.700744
1822,28/12/2021,4.0,-5.714035,86.422222,3.097778,3.038182,620.830330,914.931349,851.060055,164.700744,205.577578
1823,29/12/2021,5.0,-1.770175,91.611111,1.506667,1.430909,619.364426,878.801180,914.931349,205.577578,170.559363
1824,30/12/2021,6.0,2.092982,93.988889,1.960000,1.900000,508.063438,818.608756,878.801180,170.559363,176.970948


In [3]:
df[["day", "month", "year"]] = df["Date"].str.split("/", expand = True)
df['day'] = df['day'].astype(float)
df['month'] = df['month'].astype(float)
df['year'] = df['year'].astype(float)

In [4]:
df = df.drop(['Date'],axis=1)

In [5]:
df.values

array([[ 7.00000000e+00, -2.07894737e+00,  8.56666667e+01, ...,
         1.00000000e+00,  1.00000000e+00,  2.01700000e+03],
       [ 1.00000000e+00, -1.88596491e+00,  8.37111111e+01, ...,
         2.00000000e+00,  1.00000000e+00,  2.01700000e+03],
       [ 2.00000000e+00, -1.63157895e+00,  8.97111111e+01, ...,
         3.00000000e+00,  1.00000000e+00,  2.01700000e+03],
       ...,
       [ 5.00000000e+00, -1.77017544e+00,  9.16111111e+01, ...,
         2.90000000e+01,  1.20000000e+01,  2.02100000e+03],
       [ 6.00000000e+00,  2.09298246e+00,  9.39888889e+01, ...,
         3.00000000e+01,  1.20000000e+01,  2.02100000e+03],
       [ 7.00000000e+00,  7.37017544e+00,  9.44666667e+01, ...,
         3.10000000e+01,  1.20000000e+01,  2.02100000e+03]])

In [6]:
data = torch.tensor(df.values)
data

tensor([[ 7.0000e+00, -2.0789e+00,  8.5667e+01,  ...,  1.0000e+00,
          1.0000e+00,  2.0170e+03],
        [ 1.0000e+00, -1.8860e+00,  8.3711e+01,  ...,  2.0000e+00,
          1.0000e+00,  2.0170e+03],
        [ 2.0000e+00, -1.6316e+00,  8.9711e+01,  ...,  3.0000e+00,
          1.0000e+00,  2.0170e+03],
        ...,
        [ 5.0000e+00, -1.7702e+00,  9.1611e+01,  ...,  2.9000e+01,
          1.2000e+01,  2.0210e+03],
        [ 6.0000e+00,  2.0930e+00,  9.3989e+01,  ...,  3.0000e+01,
          1.2000e+01,  2.0210e+03],
        [ 7.0000e+00,  7.3702e+00,  9.4467e+01,  ...,  3.1000e+01,
          1.2000e+01,  2.0210e+03]], dtype=torch.float64)

In [7]:
data[0]

tensor([ 7.0000e+00, -2.0789e+00,  8.5667e+01,  2.8178e+00,  2.5873e+00,
         4.6602e+02,  5.5551e+02,  5.5551e+02,  1.7918e+02,  1.7918e+02,
         1.0000e+00,  1.0000e+00,  2.0170e+03], dtype=torch.float64)

In [8]:
import series_transformer as st

dataset = st.CustomDataSet('Avg_data.csv')

dataset[0][0]

c:\Users\trzci\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


tensor([[ 7.0000e+00, -2.0789e+00,  8.5667e+01,  2.8178e+00,  2.5873e+00,
          4.6602e+02,  5.5551e+02,  5.5551e+02,  1.7918e+02,  1.7918e+02,
          1.0000e+00,  1.0000e+00,  2.0170e+03],
        [ 1.0000e+00, -1.8860e+00,  8.3711e+01,  2.8867e+00,  2.6473e+00,
          4.8238e+02,  6.1519e+02,  5.5551e+02,  1.7918e+02,  1.8510e+02,
          2.0000e+00,  1.0000e+00,  2.0170e+03],
        [ 2.0000e+00, -1.6316e+00,  8.9711e+01,  3.6333e+00,  3.3127e+00,
          5.0712e+02,  6.2323e+02,  6.1519e+02,  1.8510e+02,  2.1847e+02,
          3.0000e+00,  1.0000e+00,  2.0170e+03],
        [ 3.0000e+00,  2.2982e-01,  8.8800e+01,  5.2578e+00,  4.6891e+00,
          4.8430e+02,  6.2604e+02,  6.2323e+02,  2.1847e+02,  2.0718e+02,
          4.0000e+00,  1.0000e+00,  2.0170e+03],
        [ 4.0000e+00, -5.0333e+00,  8.0111e+01,  4.3844e+00,  3.9745e+00,
          4.8399e+02,  6.8980e+02,  6.2604e+02,  2.0718e+02,  2.2394e+02,
          5.0000e+00,  1.0000e+00,  2.0170e+03],
        [ 5.000

In [9]:
params = st.ParameterProvider("series.config")
t1 = st.Transformer(params)

KeyError: 'DIMENSIONS AND SIZES'

In [ ]:
# to see if they are shifted
print(dataset[0][0][-1])
print(dataset[0][1][0])
print('---')
print(dataset[0][1][1])
print(dataset[0][2][0])

tensor([ 2.0000e+00, -2.3105e+00,  8.7689e+01,  1.3822e+00,  1.2273e+00,
         4.9581e+02,  6.4118e+02,  6.3894e+02,  1.6916e+02,  1.5426e+02,
         1.0000e+00,  2.0000e+00,  2.0170e+03], dtype=torch.float64)
tensor([ 2.0000e+00, -2.3105e+00,  8.7689e+01,  1.3822e+00,  1.2273e+00,
         4.9581e+02,  6.4118e+02,  6.3894e+02,  1.6916e+02,  1.5426e+02,
         1.0000e+00,  2.0000e+00,  2.0170e+03], dtype=torch.float64)
---
tensor([ 3.0000e+00, -5.0000e-01,  9.3689e+01,  1.4511e+00,  1.2400e+00,
         5.1971e+02,  6.2564e+02,  6.4118e+02,  1.5426e+02,  1.2932e+02,
         2.0000e+00,  2.0000e+00,  2.0170e+03], dtype=torch.float64)
tensor([ 3.0000e+00, -5.0000e-01,  9.3689e+01,  1.4511e+00,  1.2400e+00,
         5.1971e+02,  6.2564e+02,  6.4118e+02,  1.5426e+02,  1.2932e+02,
         2.0000e+00,  2.0000e+00,  2.0170e+03], dtype=torch.float64)


In [ ]:
torch.unsqueeze(dataset[0][0][-1],0)

tensor([[ 2.0000e+00, -2.3105e+00,  8.7689e+01,  1.3822e+00,  1.2273e+00,
          4.9581e+02,  6.4118e+02,  6.3894e+02,  1.6916e+02,  1.5426e+02,
          1.0000e+00,  2.0000e+00,  2.0170e+03]], dtype=torch.float64)

In [ ]:
print(torch.cuda.is_available())
device_id = torch.cuda.current_device()
device_id

True


0

In [ ]:
t1.cuda(device_id)

Transformer(
  (encoder_input_transformation): InputTransformation(
    (linear): Linear(in_features=13, out_features=32, bias=True)
  )
  (decoder_input_transformation): InputTransformation(
    (linear): Linear(in_features=13, out_features=32, bias=True)
  )
  (pos_encoding_in): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pos_encoding_out): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_stack): EncoderStack(
    (encoders): ModuleList(
      (0-3): 4 x EncoderLayer(
        (feed_forward): Sequential(
          (0): Linear(in_features=32, out_features=64, bias=True)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=32, bias=True)
        )
        (mha): MultiHeadedAttention(
          (heads): ModuleList(
            (0-3): 4 x AttentionHead(
              (l): Linear(in_features=100, out_features=200, bias=True)
              (WQ): Linear(in_features=32, out_features=12, bias=True)
              (WK

In [ ]:
torch.unsqueeze(dataset[0][0][-1],0).to(torch.float).dtype

torch.float32

In [ ]:
output = t1(torch.unsqueeze(dataset[0][0],0).to(torch.float).cuda(device_id), torch.unsqueeze(dataset[0][1],0).to(torch.float).cuda(device_id))

In [ ]:
output.size()

torch.Size([1, 7, 13])

In [ ]:
torch.unsqueeze(dataset[0][2],0).size()

torch.Size([1, 7, 13])